In [1]:
import ee

ee.Authenticate()


Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
REGION_NAMES = ['El Jadida', 'Gabon', 'Isiolo', 'Khouribga', 'Kitui', 'Laikipia', 'Marsabit', 'Maynas', 'Para', 'Safi', 'Selangor', 'Sri Lanka']
REGION_TYPES = [2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0]
REGION_NAMES_FS = [region.lower().replace(' ', '') for region in REGION_NAMES]
# Imports
noaa = ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5")
gaul = ee.FeatureCollection("FAO/GAUL/2015/level2")
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
pdsi = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
spei = ee.ImageCollection("CSIC/SPEI/2_8")
# variables
start_date = '2000-01-01'
end_date = '2020-01-01'
filter = ee.Filter.date(start_date, end_date)
spei_length = '01'
# function
def create_TS_spei(image, county_bounds, length):
    date = image.get('system:time_start')
    value = image.reduceRegion(ee.Reducer.mean(), county_bounds).get(f'SPEI_{length}_month')
    ft = ee.Feature(None, {'system:time_start': ee.Date(date).format("MMM dd, YYYY"), 
                            'value': value})
    return ft

In [4]:
task_list = []
for region, region_type in zip(REGION_NAMES, REGION_TYPES):
    region_printable = region.lower().replace(' ', '')
    county = gaul.filter(ee.Filter.eq(f'ADM{region_type}_NAME', region))
    county_bounds = county.geometry().bounds()
    spei_images = spei.filter(filter)
    spei_data = spei_images.map(lambda arg: create_TS_spei(arg, county_bounds, spei_length))
    task = ee.batch.Export.table.toDrive(spei_data, f'spei_{spei_length}_{region_printable}', region_printable, selectors = 'system:time_start, value',)
    task.start()
    task_list.append(task)

In [6]:
for task in task_list:
    status = task.status()
    description = status['description']
    state = status['state']
    if state != 'COMPLETED':
        print(description, state) 